# Project: The Virtual Power Plant
## Valuing Data Center Flexibility in ERCOT's Future Grid

**Course Project:** MAE/ENE 539  
**Model:** Linear Capacity Expansion with Demand Side Flexibility

### Overview
This notebook investigates the impact of adding a large (1 GW), flexible Data Center load to the ERCOT grid. Unlike traditional demand, this load is modeled as a "Virtual Power Plant"—it acts as a firm load most of the time but can "dispatch" (shut down) its consumption when grid prices exceed its opportunity cost (Strike Price).

### Scenarios to Test
1.  **Baseline (Firm Load):** The Data Center must run 24/7 (Strike Price = $9,000/MWh).
2.  **Flexible Load (Virtual Battery):** The Data Center allows interruption for a fee (Strike Price = $200 - $1,000/MWh).
3.  **Locational Value:** Siting the Data Center in Zone 1 (West Texas Wind) vs. Zone 2 (Urban Demand Center).

In [1]:
using JuMP
using HiGHS
using DataFrames, CSV
using Statistics

## 1. Load Data (16-Week Resolution)
We use the higher-resolution 16-week dataset to capture seasonal volatility.

In [2]:
# Define path to the 16-week dataset
inputs_path = "ercot_brownfield_expansion/16_weeks/"

# LOAD GENERATORS & STORAGE
generators = DataFrame(CSV.File(joinpath(inputs_path, "Generators_data.csv")))
# Filter columns to keep model lightweight
generators = select(generators, :R_ID, :Resource, :zone, :THERM, :DISP, :NDISP, :STOR, :HYDRO, :RPS, :CES,
                    :Commit, :Existing_Cap_MW, :Existing_Cap_MWh, :Cap_size, :New_Build, :Max_Cap_MW,
                    :Inv_cost_per_MWyr, :Fixed_OM_cost_per_MWyr, :Inv_cost_per_MWhyr, :Fixed_OM_cost_per_MWhyr,
                    :Var_OM_cost_per_MWh, :Start_cost_per_MW, :Start_fuel_MMBTU_per_MW, :Heat_rate_MMBTU_per_MWh, :Fuel,
                    :Min_power, :Ramp_Up_percentage, :Ramp_Dn_percentage, :Up_time, :Down_time,
                    :Eff_up, :Eff_down);

# LOAD DEMAND & NETWORK
demand_inputs = DataFrame(CSV.File(joinpath(inputs_path, "Load_data.csv")))
variability = DataFrame(CSV.File(joinpath(inputs_path, "Generators_variability.csv")))
variability = variability[:,2:ncol(variability)] # Drop index column
fuels = DataFrame(CSV.File(joinpath(inputs_path, "Fuels_data.csv")))
network = DataFrame(CSV.File(joinpath(inputs_path, "Network.csv")))

println("Data Loaded Successfully.")

Data Loaded Successfully.


## 2. SCENARIO SETUP: The "Virtual Power Plant"
**Instructions:** Modify the variables below to test different scenarios.
* To test **Firm Load**, set `interruption_strike_price = 9000` (VOLL).
* To test **Flexible Load**, set `interruption_strike_price = 500`.

In [3]:
# === USER INPUTS ===
data_center_mw = 1000.0         # Size of the new load
data_center_zone = 1            # Zone 1 (West TX) or Zone 2 (East TX)
interruption_strike_price = 300.0 # $/MWh (The "cost" to shut down)

# === MODELING LOGIC ===

# 1. Add Flat Load to Demand
# Note: We select specific columns to avoid modifying the original 'demand_inputs' incorrectly if re-run
demand = select(demand_inputs, :Load_MW_z1, :Load_MW_z2, :Load_MW_z3)
if data_center_zone == 1
    demand.Load_MW_z1 .+= data_center_mw
elseif data_center_zone == 2
    demand.Load_MW_z2 .+= data_center_mw
elseif data_center_zone == 3
    demand.Load_MW_z3 .+= data_center_mw
end
println("Added $(data_center_mw) MW flat load to Zone $(data_center_zone).")

# 2. Prepare Generators DataFrame
# Convert R_ID to String so we can add a text ID like "DataCenter_DR"
generators.R_ID = string.(generators.R_ID) 

# 3. Create "Virtual Generator" (Demand Response Resource)
# We use Dict() to create a MUTABLE copy of the first row
new_dr = Dict(pairs(generators[1, :]))

# Update fields using [:Symbol] notation for the Dict
new_dr[:R_ID] = "DataCenter_DR" 
new_dr[:Resource] = "Demand_Response"
new_dr[:zone] = data_center_zone
new_dr[:THERM] = 0; new_dr[:DISP] = 1; new_dr[:STOR] = 0; new_dr[:HYDRO] = 0
new_dr[:Existing_Cap_MW] = data_center_mw 
new_dr[:Max_Cap_MW] = data_center_mw      
new_dr[:New_Build] = 0                    
new_dr[:Var_OM_cost_per_MWh] = interruption_strike_price
new_dr[:Fuel] = "DR_Virtual"
new_dr[:Heat_rate_MMBTU_per_MWh] = 0.0
new_dr[:Min_power] = 0.0
new_dr[:Ramp_Up_percentage] = 1.0 
new_dr[:Ramp_Dn_percentage] = 1.0

# Clean up other cost columns to be zero
new_dr[:Inv_cost_per_MWyr] = 0.0
new_dr[:Fixed_OM_cost_per_MWyr] = 0.0
new_dr[:Start_cost_per_MW] = 0.0

# Add to generators list
push!(generators, new_dr)
G = generators.R_ID

# FIX: Added a backslash (\) before the first $ so it prints as text
println("Created 'DataCenter_DR' resource with strike price \$$(interruption_strike_price)/MWh.")

Added 1000.0 MW flat load to Zone 1.
Created 'DataCenter_DR' resource with strike price $300.0/MWh.


## 3. Pre-Processing (Calculations)

In [4]:
# =================================================================
# 3. PRE-PROCESSING
# =================================================================

# --- A. Original Calculations (KEEP THIS) ---

# Process Time/Sample Weights
hours_per_period = convert(Int64, demand_inputs.Hours_per_period[1])
P = convert(Array{Int64}, 1:demand_inputs.Subperiods[1])
W = convert(Array{Int64}, collect(skipmissing(demand_inputs.Sub_Weights)))
T = convert(Array{Int64}, demand_inputs.Time_index)

sample_weight = zeros(Float64, size(T,1))
t=1
for p in P
    for h in 1:hours_per_period
        # FIX: Only fill weights if we are still within the (sliced) time horizon
        if t <= length(T)
            sample_weight[t] = W[p]/hours_per_period
            t=t+1
        end
    end
end

# Process Variable Costs & CO2
generators.Var_Cost = zeros(Float64, size(G,1))
for g in 1:nrow(generators)
    # If it is the DR resource, cost is just the OM cost (Strike Price)
    if generators.Fuel[g] == "DR_Virtual"
        generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g]
    else
        # Normal Generators: Fuel * HeatRate + VOM
        fuel_cost = fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]
        generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g] + 
                                 fuel_cost * generators.Heat_rate_MMBTU_per_MWh[g]
    end
end

# Define Sets
S = convert(Array{Int64}, collect(skipmissing(demand_inputs.Demand_segment)))
Z = convert(Array{Int64}, 1:3)
zones = collect(skipmissing(network.Network_zones))
lines = select(network[1:2,:], :Network_lines, :z1, :z2, :z3, :Line_Max_Flow_MW, 
               :Line_Reinforcement_Cost_per_MW_yr)
lines.Line_Fixed_Cost_per_MW_yr = lines.Line_Reinforcement_Cost_per_MW_yr./20
L = convert(Array{Int64}, lines.Network_lines)

# NSE Data (Value of Lost Load)
VOLL = demand_inputs.Voll[1]
nse = DataFrame(Segment=S, 
                NSE_Cost = VOLL.*collect(skipmissing(demand_inputs.Cost_of_demand_curtailment_perMW)),
                NSE_Max = collect(skipmissing(demand_inputs.Max_demand_curtailment)))

# Subsets
STOR = intersect(generators.R_ID[generators.STOR.>=1], G)
NEW = intersect(generators.R_ID[generators.New_Build.==1], G)
OLD = intersect(generators.R_ID[.!(generators.New_Build.==1)], G)


# --- B. New Fixes for String IDs (ADD THIS) ---

# 1. Sync Variability Columns with Generator IDs
# We need to rename the variability columns to match the IDs ("1", "2", etc.)
# so we can access them by name (e.g., variability[t, "1"]).

# Identify how many original generators we have (excluding the new Data Center)
num_original_gens = ncol(variability)

# Get the IDs for those original generators (first N rows)
original_ids = generators.R_ID[1:num_original_gens]

# Rename the variability columns to match these IDs
rename!(variability, Symbol.(original_ids))

# Now add the new "DataCenter_DR" column to variability
if "DataCenter_DR" ∉ names(variability)
    variability[!, "DataCenter_DR"] .= 1.0 # Available 100% of the time
end

# 2. Create a Generator Lookup Dictionary
# This is CRITICAL. It maps the String ID ("DataCenter_DR") to the entire row of data.
# This allows the model to find cost/capacity data using the ID.
gen_lookup = Dict(row.R_ID => row for row in eachrow(generators))

println("Pre-Processing Complete. Variability columns renamed and lookup dict created.")

Pre-Processing Complete. Variability columns renamed and lookup dict created.


## 4. Optimization Model

In [5]:
# =================================================================
# 4. OPTIMIZATION MODEL
# =================================================================

Expansion_Model = Model(HiGHS.Optimizer)

# --- VARIABLES ---
# No changes needed here; JuMP handles string sets (G) automatically
@variables(Expansion_Model, begin
    vCAP[g in G] >= 0
    vNEW_CAP[g in NEW] >= 0
    vRET_CAP[g in OLD] >= 0
    vE_CAP[g in STOR] >= 0
    vNEW_E_CAP[g in intersect(STOR, NEW)] >= 0
    vRET_E_CAP[g in intersect(STOR, OLD)] >= 0
    vT_CAP[l in L] >= 0
    vNEW_T_CAP[l in L] >= 0
    vGEN[T,G] >= 0
    vCHARGE[T,STOR] >= 0
    vSOC[T,STOR] >= 0
    vNSE[T,S,Z] >= 0
    vFLOW[T,L]
end)

# --- CONSTRAINTS ---

# 1. Demand Balance
@constraint(Expansion_Model, cDemandBalance[t in T, z in Z],
    sum(vGEN[t,g] for g in intersect(generators[generators.zone.==z,:R_ID],G)) +
    sum(vNSE[t,s,z] for s in S) - 
    sum(vCHARGE[t,g] for g in intersect(generators[generators.zone.==z,:R_ID],STOR)) -
    demand[t,z] - 
    sum(lines[l,Symbol(string("z",z))] * vFLOW[t,l] for l in L) == 0
)

# 2. Generator Constraints
# FIX: Use 'gen_lookup[g]' instead of 'generators[g]'
for g in NEW
    # We check the dictionary for the Max Cap value
    if gen_lookup[g].Max_Cap_MW > 0
        set_upper_bound(vNEW_CAP[g], gen_lookup[g].Max_Cap_MW)
    end
end

# Operational Limits
# FIX: variability[t, g] works now because we renamed columns in Sec 3.
# FIX: gen_lookup[g].Property replaces generators.Property[g]
@constraints(Expansion_Model, begin
    cMaxPower[t in T, g in G], vGEN[t,g] <= variability[t, g] * vCAP[g]
    cCapOld[g in OLD], vCAP[g] == gen_lookup[g].Existing_Cap_MW - vRET_CAP[g]
    cCapNew[g in NEW], vCAP[g] == vNEW_CAP[g]
end)

# Storage Constraints
@constraints(Expansion_Model, begin
    cMaxCharge[t in T, g in STOR], vCHARGE[t,g] <= vCAP[g]
    cMaxSOC[t in T, g in STOR], vSOC[t,g] <= vE_CAP[g]
    cCapEnergyOld[g in intersect(STOR, OLD)], vE_CAP[g] == gen_lookup[g].Existing_Cap_MWh - vRET_E_CAP[g]
    cCapEnergyNew[g in intersect(STOR, NEW)], vE_CAP[g] == vNEW_E_CAP[g]
end)

# Transmission Constraints
@constraints(Expansion_Model, begin
    cTransCap[l in L], vT_CAP[l] == lines.Line_Max_Flow_MW[l] + vNEW_T_CAP[l]
    cMaxFlow[t in T, l in L], vFLOW[t,l] <= vT_CAP[l]
    cMinFlow[t in T, l in L], vFLOW[t,l] >= -vT_CAP[l]
end)

# Time Coupling
STARTS = 1:hours_per_period:maximum(T)
INTERIORS = setdiff(T,STARTS)

@constraints(Expansion_Model, begin
    cRampUp[t in INTERIORS, g in G], 
        vGEN[t,g] - vGEN[t-1,g] <= gen_lookup[g].Ramp_Up_percentage * vCAP[g]
    
    cSOC[t in INTERIORS, g in STOR], 
        vSOC[t,g] == vSOC[t-1,g] + gen_lookup[g].Eff_up * vCHARGE[t,g] - vGEN[t,g] / gen_lookup[g].Eff_down
end)

# --- OBJECTIVE ---
# FIX: All cost lookups now use gen_lookup[g].CostParameter
@objective(Expansion_Model, Min,
    sum(gen_lookup[g].Fixed_OM_cost_per_MWyr * vCAP[g] for g in G) +
    sum(gen_lookup[g].Inv_cost_per_MWyr * vNEW_CAP[g] for g in NEW) +
    sum(gen_lookup[g].Fixed_OM_cost_per_MWhyr * vE_CAP[g] for g in STOR) +
    sum(gen_lookup[g].Inv_cost_per_MWhyr * vNEW_E_CAP[g] for g in intersect(STOR, NEW)) +
    sum(lines.Line_Fixed_Cost_per_MW_yr[l] * vT_CAP[l] + lines.Line_Reinforcement_Cost_per_MW_yr[l] * vNEW_T_CAP[l] for l in L) +
    sum(sample_weight[t] * gen_lookup[g].Var_Cost * vGEN[t,g] for t in T, g in G) +
    sum(sample_weight[t] * nse.NSE_Cost[s] * vNSE[t,s,z] for t in T, s in S, z in Z)
)

45062 vCAP[1] + 890411 vCAP[2] + 71207 vCAP[3] + 41917 vCAP[4] + 13642 vCAP[5] + 8670 vCAP[6] + 47301 vCAP[7] + 105165 vCAP[8] + 45062 vCAP[9] + 101453 vCAP[10] + 18715 vCAP[11] + 42036 vCAP[12] + 13642 vCAP[13] + 7136 vCAP[14] + 45062 vCAP[15] + 18715 vCAP[16] + 35065 vCAP[17] + 10463 vCAP[18] + 7143 vCAP[19] + 38186 vCAP[20] + 38186 vCAP[21] + 38186 vCAP[22] + 12603 vCAP[23] + 12603 vCAP[24] + 12603 vCAP[25] + 12603 vCAP[26] + 8557 vCAP[27] + 44735 vCAP[28] + 35065 vCAP[29] + 10463 vCAP[30] + [[...75308 terms omitted...]] + 9000 vNSE[2684,1,1] + 9000 vNSE[2684,1,2] + 9000 vNSE[2684,1,3] + 603 vNSE[2684,2,1] + 603 vNSE[2684,2,2] + 603 vNSE[2684,2,3] + 9000 vNSE[2685,1,1] + 9000 vNSE[2685,1,2] + 9000 vNSE[2685,1,3] + 603 vNSE[2685,2,1] + 603 vNSE[2685,2,2] + 603 vNSE[2685,2,3] + 9000 vNSE[2686,1,1] + 9000 vNSE[2686,1,2] + 9000 vNSE[2686,1,3] + 603 vNSE[2686,2,1] + 603 vNSE[2686,2,2] + 603 vNSE[2686,2,3] + 9000 vNSE[2687,1,1] + 9000 vNSE[2687,1,2] + 9000 vNSE[2687,1,3] + 603 vNSE[2687,2

## 5. Solve and Analyze

In [8]:
# =================================================================
# 5. RESULTS ANALYSIS
# =================================================================

println("Solving Model... This may take a few minutes.")
@time optimize!(Expansion_Model)

Solving Model... This may take a few minutes.
LP   has 343181 rows; 188282 cols; 1006144 nonzeros
Coefficient ranges:
  Matrix [1e-04, 4e+00]
  Cost   [1e+01, 9e+05]
  Bound  [8e+02, 9e+04]
  RHS    [1e+01, 9e+04]
Solving LP without presolve, or with basis, or unconstrained
Model status        : Optimal
Objective value     :  1.4400780025e+10
P-D objective error :  2.7151756295e-15
HiGHS run time      :          0.54
  0.553396 seconds (9 allocations: 272 bytes)


In [9]:
# --- A. Data Center Analysis ---
# We use a list comprehension to safely extract values for the Data Center
dc_gen = value.(vGEN)[:, "DataCenter_DR"]
dc_gen_vec = [dc_gen[t] for t in T]

# Calculate totals
total_dc_curtailment_MWh = sum(sample_weight .* dc_gen_vec)
total_dc_hours_curtailed = count(x -> x > 1.0, dc_gen_vec)

println("--- DATA CENTER RESULTS ---")
println("Scenario Strike Price: \$$interruption_strike_price / MWh")
println("Total Load Curtailed (Virtual Generation): $(round(total_dc_curtailment_MWh, digits=0)) MWh")
println("Number of Hours Curtailed: $total_dc_hours_curtailed hours")


# --- B. Capacity Expansion Analysis ---
# FIX: Use List Comprehensions + gen_lookup to handle String IDs safely.
# This avoids the "unable to check bounds" error.

new_builds = DataFrame(
    ID = NEW,
    # Look up the name in our dictionary
    Resource = [gen_lookup[g].Resource for g in NEW], 
    # Extract the value from the variable directly
    New_MW = [value(vNEW_CAP[g]) for g in NEW]
)

# Only show what was actually built (> 1 MW)
filter!(row -> row.New_MW > 1.0, new_builds)

println("\n--- NEW CAPACITY BUILT ---")
show(new_builds, allrows=true)

--- DATA CENTER RESULTS ---
Scenario Strike Price: $300.0 / MWh
Total Load Curtailed (Virtual Generation): 32255.0 MWh
Number of Hours Curtailed: 23 hours

--- NEW CAPACITY BUILT ---
4×3 DataFrame
 Row │ ID      Resource              New_MW     
     │ String  String                Float64    
─────┼──────────────────────────────────────────
   1 │ 23      utilitypv_losangeles    157.848
   2 │ 30      naturalgas_ccavgcf     6584.02
   3 │ 38      utilitypv_losangeles  19057.2
   4 │ 52      utilitypv_losangeles     98.4579